In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

# 2022

In [3]:
team_stats = pd.read_csv('../footy_data/league_one_teams_2022.csv')
games = pd.read_csv('../footy_data/league_one_games_2022.csv')

In [4]:
games = games.loc[games['status'] == 'complete']
games
#Only include completed games

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Bolton Wanderers,Milton Keynes Dons,Martin Coy,1,0.00,0.00,...,2.45,3.35,2.75,1.29,1.80,2.80,4.95,1.69,2.15,University of Bolton Stadium (Bolton)
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Cambridge United,Oxford United,David Rock,1,0.00,0.00,...,3.55,3.40,2.05,1.35,1.95,3.10,5.50,1.80,2.00,Abbey Stadium (Cambridge- Cambridgeshire)
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Crewe Alexandra,Cheltenham Town,Ben Toner,1,0.00,0.00,...,2.25,3.05,3.40,1.39,2.10,3.40,6.75,1.87,1.91,Mornflake Stadium (Crewe- Cheshire)
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Doncaster Rovers,AFC Wimbledon,Samuel Barrott,1,0.00,0.00,...,2.40,3.25,2.95,1.34,1.95,3.10,5.75,1.77,2.00,The Keepmoat Stadium (Doncaster- South Yorkshire)
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Fleetwood Town,Portsmouth,Ben Speedie,1,0.00,0.00,...,2.95,3.15,2.40,1.48,2.25,3.90,7.50,2.05,1.74,Highbury Stadium (Fleetwood- Lancashire)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1631368800,Sep 11 2021 - 2:00pm,complete,NaN,Plymouth Argyle,Sheffield Wednesday,Ben Speedie,7,1.67,1.33,...,3.25,3.40,2.15,1.43,2.15,3.70,7.00,2.00,1.77,Home Park (Plymouth)
75,1631368800,Sep 11 2021 - 2:00pm,complete,NaN,Rotherham United,Fleetwood Town,Craig Hicks,7,2.00,0.33,...,1.61,3.90,5.25,1.38,2.15,3.70,7.25,2.20,1.65,AESSEAL New York Stadium (Rotherham- South Yor...
76,1631368800,Sep 11 2021 - 2:00pm,complete,NaN,Shrewsbury Town,Crewe Alexandra,Samuel Barrott,7,1.00,0.00,...,2.30,3.15,3.10,1.43,2.20,3.65,7.00,2.00,1.80,Montgomery Waters Meadow (Shrewsbury- Shropshire)
77,1631368800,Sep 11 2021 - 2:00pm,complete,NaN,Sunderland,Accrington Stanley,Anthony Backhouse,7,3.00,1.00,...,1.71,3.70,4.75,1.31,1.83,2.95,5.25,1.83,1.95,Stadium of Light (Sunderland)


Seeing how many games had goals and xg of over 2.5

In [5]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.99, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['over/under_1.5'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 1.5, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [6]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100
percent_goals_1h

47.05882352941176

In [7]:
games['over/under'].value_counts()

0    43
1    33
Name: over/under, dtype: int64

In [8]:
games['xg_over/under'].value_counts()

0    53
1    23
Name: xg_over/under, dtype: int64

In [9]:
xg_view = games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]

In [11]:
xg_view[50:85]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
50,Cambridge United,Bolton Wanderers,0.98,1.66,1,0
51,Charlton Athletic,Crewe Alexandra,2.25,1.66,2,0
52,Ipswich Town,AFC Wimbledon,1.48,1.22,2,2
53,Milton Keynes Dons,Accrington Stanley,1.41,1.37,2,0
54,Morecambe,Sheffield Wednesday,0.85,1.46,1,0
55,Oxford United,Lincoln City,1.75,1.35,3,1
56,Plymouth Argyle,Fleetwood Town,1.62,0.91,1,1
57,Shrewsbury Town,Gillingham,1.61,1.18,2,1
58,Sunderland,Wycombe Wanderers,1.24,2.07,3,1
59,Wigan Athletic,Portsmouth,0.94,1.32,1,0


In [30]:
first_matchweek = games[0:12]

In [31]:
first_matchweek['over/under'].value_counts()

0    7
1    5
Name: over/under, dtype: int64

In [32]:
first_matchweek['over/under_1.5'].value_counts()

1    9
0    3
Name: over/under_1.5, dtype: int64

In [33]:
games['over/under_1.5'].value_counts()

1    53
0    23
Name: over/under_1.5, dtype: int64

In [34]:
games['over/under'].value_counts()

0    43
1    33
Name: over/under, dtype: int64

In [35]:
games['total_goals_at_half_time'].value_counts()

2    24
0    24
1    23
3     4
5     1
Name: total_goals_at_half_time, dtype: int64

In [36]:
games['home_team_goal_count_half_time'].sum()

49

In [37]:
games['away_team_goal_count_half_time'].sum()

39

# Teams

In [18]:
team_stats[['team_name', 'xg_for_avg_overall', 'xg_against_avg_overall']]

,team_name,xg_for_avg_overall,xg_against_avg_overall
0,Sunderland AFC,1.29,1.02
1,Burton Albion FC,1.19,1.37
2,Rotherham United FC,1.35,0.55
3,Sheffield Wednesday FC,0.76,0.77
4,Ipswich Town FC,1.05,1.00
5,Wigan Athletic FC,0.71,0.96
6,Charlton Athletic FC,0.96,0.99
7,Bolton Wanderers FC,1.54,1.14
8,Milton Keynes Dons FC,1.44,0.97
9,Doncaster Rovers FC,0.53,1.22


sos = total average of xg for xg against of opponents they have played

# Adding More Columns

In [ ]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

In [ ]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats

In [ ]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [ ]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [ ]:
fill_stats(team_stats['common_name'][0])
fill_stats(team_stats['common_name'][1])
fill_stats(team_stats['common_name'][2]) 
fill_stats(team_stats['common_name'][3]) 
fill_stats(team_stats['common_name'][4]) 
fill_stats(team_stats['common_name'][5]) 
fill_stats(team_stats['common_name'][6]) 
fill_stats(team_stats['common_name'][7]) 
fill_stats(team_stats['common_name'][8]) 
fill_stats(team_stats['common_name'][9]) 
fill_stats(team_stats['common_name'][10]) 
fill_stats(team_stats['common_name'][11]) 
fill_stats(team_stats['common_name'][12]) 
fill_stats(team_stats['common_name'][13]) 
fill_stats(team_stats['common_name'][14]) 
fill_stats(team_stats['common_name'][15]) 
fill_stats(team_stats['common_name'][16]) 
fill_stats(team_stats['common_name'][17]) 
fill_stats(team_stats['common_name'][18]) 
fill_stats(team_stats['common_name'][19]) 
fill_stats(team_stats['common_name'][20]) 
fill_stats(team_stats['common_name'][21]) 
fill_stats(team_stats['common_name'][22]) 
fill_stats(team_stats['common_name'][23]) 